# Run z-brains analyses for 3T and 7T cases

In [75]:
import pandas as pd
import os, sys, subprocess
import importlib

# import utilities
utilities_path = os.path.abspath(os.path.join(os.getcwd(), "/host/verges/tank/data/daniel", "Utils"))
sys.path.append(utilities_path) # path to Utilities
import id, gen, zbUtils

In [76]:
importlib.reload(zbUtils)

<module 'zbUtils' from '/host/verges/tank/data/daniel/3T7T/z/code/run/zbUtils.py'>

In [77]:
# Input IDs

# IDs: should be in long format (all IDs to be used should be in same column (regardless of if it is 3T or 7T ID))
# model: "/host/verges/tank/data/daniel/3T7T/z/data/models/participants_mics_hc.csv"

# list col names
IDs = {
    "dir": "/host/verges/tank/data/daniel/3T7T/z/data/pt/master_demo_19Mar2025_age_y.csv",
    "ID_col": "ID",
    "SES_col": "SES",
    "age_col": "age_y",
    "sex": "sex"
}

# Dir tmp
dir_tmp = "/host/verges/tank/data/daniel/3T7T/z/outputs/tmp"

In [78]:
# zBrain parameters

# size of kernels for smoothing (cortex: ctx; hippocampus: hip). 
# NOTE: Both lists must be of same length.
smooth_ctx=[2,5,10] # try with 2,5,10
smooth_hip=[1,2,5] # try with 1,2,5

# surface
res="high" # low (cotrex: 5K, hippocampus: 2mm), high (ctx: 32k, hipp: 0.5mm), all (both high and low)

# Output: path and file name
out_name="DM_zb_37comp" # will create folders with this name in BIDs directory

# path to z-brains software
score_type = "z" # z or w, will call appropriate shell script accordingly

# Dictionary item for each study
# Note: zBrains only currently supports one 'ctrl_ptrn', not a list

# directories need to be relative except for root
MICs = {
    "name" : "MICs",
    "ctrl_ptrn" : "HC",
    "pt_ptrn" : "PX",
    "dir_root" : "/data/mica3/BIDS_MICs",
    "dir_raw" : "rawdata",
    "dir_deriv" : "derivatives",
    "dir_mp" : "micapipe_v0.2.0",
    "dir_hu" : "hippunfold_v1.3.0"
}

PNI = {
    "name" : "PNI",
    "ctrl_ptrn" : "PNC",
    "pt_ptrn" : "PNE",
    "dir_root" : "/data/mica3/BIDS_PNI",
    "dir_raw" : "rawdata",
    "dir_deriv" : "derivatives",
    "dir_mp" : "micapipe_v0.2.0",
    "dir_hu" : "hippunfold_v1.3.0"
}

studies = [MICs, PNI]

In [79]:
# testing values
test = False
if test:
    dir_software = "/host/verges/tank/data/daniel/3T7T/z/code/run/tests/printArgs.sh"
elif score_type == "z":
    dir_software = "/host/verges/tank/data/daniel/3T7T/z/code/run/runZB.sh"
elif score_type == "w":
    dir_software = "/host/verges/tank/data/daniel/3T7T/z/code/run/runZB_w.sh"
else:  
    print("Invalid score type. Must be either 'z', 'w' or 'test'")

# studies = [MICs]
# smooth_ctx=[10]
# smooth_hip=[5]
# run_arg = "analysis" # for testing, can not be defined for actual run

In [80]:
# Convert IDs to format accepted by zBrains (add sub-, ses-)

# ensure that ID and ses are properly formatted
#df = gen.fmt(IDs["dir"], [dir_IDs_cols["7T_ID"], dir_IDs_cols["3T_ID"], dir_IDs_cols["7T_SES"], dir_IDs_cols["3T_SES"]])
df = gen.fmt(IDs["dir"], ["ID", "SES"])
# rename df columns
df = df.rename(columns={IDs["ID_col"]:"ID", IDs["SES_col"]:"SES", IDs["age_col"]:"AGE", IDs["sex"]:"SEX"})
#df.head()

# save df to tmp, use path as new IDs["dir"]
dir_IDs = dir_tmp+"/"+os.path.splitext(os.path.basename(IDs["dir"]))[0]+"_fmt.csv"
df.to_csv(dir_IDs, index=False)

print("[INFO] Formatted IDs saved to: ", dir_IDs)

# seperate IDs by study - pass seperate IDs csv to zBrains
# extract ID patterns
ptrns = []
for study in studies:
    ptrns.append([f"sub-{study['ctrl_ptrn']}", f"sub-{study['pt_ptrn']}"])

#print(ptrns)

split_paths = id.split(dir_IDs, col="ID", patterns=ptrns, pth_out=dir_tmp) # split_paths holds paths to split files

# print(split_paths)
# length of split_paths should be equal to the number of studies

if len(split_paths) != len(studies):
    print("[main] Error: Number of split files does not match number of studies.")
    print("\t Check  that the attributes 'ctrl_ptrn' and 'pt_ptrn' are properly defined in all study dictionaries.")
    sys.exit(1)

[fmt] ID formatted
[fmt] SES formatted
[INFO] Formatted IDs saved to:  /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/master_demo_19Mar2025_age_y_fmt.csv
[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_1.csv (24 rows)sub-PX: 16, sub-HC: 8
[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/split_ID_2.csv (24 rows)sub-PNE: 16, sub-PNC: 8


In [82]:
# Call zbrains - can be parallelized
for i in [1,len(studies)]:
    df_path = split_paths[i-1] # demo file
    study = studies[i-1]
    ctrl_ptrn = "sub-%s" %study["ctrl_ptrn"]
    pt_ptrn = "sub-%s" %study["pt_ptrn"]
    ptrns = [ctrl_ptrn,pt_ptrn]
    #print(ptrns)

    demo_pth = id.split(df_path, col="ID", patterns=ptrns, pth_out=dir_tmp, output_prefix="demo")# filter for only control IDs  
    
    print("\n=====================================")
    print(study["name"])
    #--control_prefix and define according to study

    # run z-brains with each smoothing kernel - can run cortex and hippocampus seperately and in parallel
    for ctx, hip in zip(smooth_ctx, smooth_hip):
        print(f"Smoothing: ctx={ctx}, hip={hip}")
        
        # call via bash, pass appropriate arguments
        print("Calling zBrains with:")
        print(study, out_name, ctx, hip, res, demo_pth[0], demo_pth[1], score_type, dir_software, run_arg)
        
        result = zbUtils.runZBrain(
            study, out_name, ctx, hip, res, demo_ref=demo_pth[0], demo=demo_pth[1], 
            score=score_type, dir_software=dir_software
        )
        # Print zBrains run output
        if result.returncode != 0:
            print(f"Command failed with return code {result.returncode}")

        print("STDOUT:")
        for line in result.stdout.splitlines():
            print(f"\t{line}")

        if result.stderr:
            print("STDERR:")
            for line in result.stderr.splitlines():
                print(f"\t{line}")


[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/demo_1.csv (8 rows)sub-HC: 8
[split] Saved: /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/demo_2.csv (16 rows)sub-PX: 16

MICs
Smoothing: ctx=2, hip=1
Calling zBrains with:
{'name': 'MICs', 'ctrl_ptrn': 'HC', 'pt_ptrn': 'PX', 'dir_root': '/data/mica3/BIDS_MICs', 'dir_raw': 'rawdata', 'dir_deriv': 'derivatives', 'dir_mp': 'micapipe_v0.2.0', 'dir_hu': 'hippunfold_v1.3.0'} DM_zb_37comp 2 1 high /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/demo_1.csv /host/verges/tank/data/daniel/3T7T/z/outputs/tmp/demo_2.csv z /host/verges/tank/data/daniel/3T7T/z/code/run/runZB.sh analysis


KeyboardInterrupt: 